<a href="https://colab.research.google.com/github/AshvinVignesh/Final_year/blob/main/final_year_DL_Conv%26BiLstm_and_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install imblearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 kB 4.8 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout,Input , Layer , Bidirectional ,Reshape,MaxPooling2D, Conv1D, MaxPooling1D,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K


In [ ]:
import pandas as pd
import gensim
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report , f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
df = pd.read_csv("Full_data_aspect_removed_words.csv")
df.head()

,original_text,sentences_with_pos_tags,lang,sentiment,clean_text,Acting,direction,Music,VFX,Genre,excitement,ovr_sent,stop_words
0,இரைச்சல் இல்லாத தெளிவான இசை,இரைச்சல்/NOUN இல்லாத/ADJ தெளிவான/ADJ இசை/NOUN,Tamil,POSITIVE,இரைச்சல் இல்லாத தெளிவான இசை,0,0,1,0,0,0,1,"இல்லாத, தெளிவான"
1,harris romba naal kalichu super song,harris/PROPN romba/ADV naal/NOUN kalichu/VERB ...,Code-Mixed,POSITIVE,harris romba naal kalichu super song,0,0,1,0,0,0,1,"romba, naal, kalichu"
2,hukum pondra iraichal padalai ketu kaathu vali...,hukum/PROPN pondra/ADP iraichal/NOUN padalai/N...,Code-Mixed,NEUTRAL,hukum pondra iraichal padalai ketu kaathu vali...,0,0,1,0,0,0,1,"pondr, iraichal, ketu, kaathu, valichuruchu, a..."
3,youtube la kuda editing nalla pannuvanga,youtube/PROPN la/ADP kuda/ADV editing/NOUN nal...,Code-Mixed,NEUTRAL,youtube kuda editing nalla pannuvanga,0,0,0,1,0,0,1,"la, kuda"
4,amaran is also a diwali release,amaran/PROPN is/AUX also/ADV a/DET diwali/PROP...,Code-Mixed,NEUTRAL,amaran diwali release,0,0,0,0,0,0,0,"is, also, a"


In [ ]:
X = df['clean_text']

In [ ]:
y = df[ ['Acting', 'direction', 'Music', 'Genre', 'excitement', 'ovr_sent']]

In [ ]:
def prepare_data(df, aspect_column):
    """Prepare data for a specific aspect"""
    # Encode text to numerical values if needed
    # le = LabelEncoder()
    # Changed to return a DataFrame instead of a Series
    X = df[['clean_text']]
    y = df[aspect_column]
    return X, y

In [ ]:
X,y = prepare_data(df,'Acting')

In [ ]:
embeddings_index = {}
f = open('/content/ft_300_fyp_150epoch_window2.txt')
for line in f:
    values = line.split()
    word = values[0] # Since the word is at the 0th index
    coefs = np.asarray(values[1:]) # From the 1st position onwards the values are the embedding values
    embeddings_index[word] = coefs
f.close()

In [ ]:
EMBEDDING_DIM =300

# words not found in embedding index will be all-zeros.

In [ ]:
def analyze_all_aspects_dl(df, aspects, sampling_techniques):
    results = {}
    for aspect in aspects:
        print(f"\nAnalyzing aspect: {aspect}")
        X, y = prepare_data(df, aspect)
        aspect_results = {}
        for technique in sampling_techniques:
            print(f"\nUsing {technique} sampling:")
            mean_score, std_score, reports = train_evaluate_dl_model(X, y, technique)
            aspect_results[technique] = {
                'mean_accuracy': mean_score,
                'std_accuracy': std_score,
                'classification_reports': reports
            }
            print(f"Mean Accuracy: {mean_score:.4f} (\u00b1{std_score:.4f})")
        results[aspect] = aspect_results
    return results

In [ ]:
def train_evaluate_dl_model(X, y, sampling_strategy='none'):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    reports = []

    # Tokenize and pad sequences
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X['clean_text'])
    sequences = tokenizer.texts_to_sequences(X['clean_text'])
    padded_sequences = pad_sequences(sequences, maxlen=20)
    word_index = tokenizer.word_index
    num_words = len(word_index) + 1
    embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # Check if the embedding vector has the expected dimension
            if len(embedding_vector) == EMBEDDING_DIM:
                embedding_matrix[i] = embedding_vector
            else:
                print(f"Warning: Embedding for '{word}' has unexpected dimension {len(embedding_vector)}. Skipping.")
    # Encode labels
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    target_names = [str(class_label) for class_label in le.classes_]
    # Prepare embedding matrix (assuming it's preloaded as `embedding_matrix`)
    vocab_size = len(word_index) + 1

    for fold, (train_idx, val_idx) in enumerate(skf.split(padded_sequences, y_encoded)):
        X_train, X_val = padded_sequences[train_idx], padded_sequences[val_idx]
        y_train, y_val = y_encoded[train_idx], y_encoded[val_idx]

        # Apply undersampling if required
        if sampling_strategy == 'undersample':
            sampler = RandomUnderSampler(random_state=42)
            X_train_flat = X_train.reshape(X_train.shape[0], -1)
            X_train_flat, y_train = sampler.fit_resample(X_train_flat, y_train)
            X_train = X_train_flat.reshape(-1, 20)  # Reshape back
        elif sampling_strategy == 'oversample':
            sampler = SMOTE(random_state=42)
            # Flatten `X_train` for SMOTE
            X_train_flat = X_train.reshape(X_train.shape[0], -1)
            X_train_flat, y_train = sampler.fit_resample(X_train_flat, y_train)
            # Reshape back to the original dimensions
            X_train = X_train_flat.reshape(-1, 20)

        # Define the model
        input_layer = tf.keras.layers.Input(shape=(20,))
        embedding_layer = tf.keras.layers.Embedding(
            vocab_size, 300, weights=[embedding_matrix], input_length=20, trainable=False
        )(input_layer)
        conv = tf.keras.layers.Conv1D(200, kernel_size=3, activation="relu")(embedding_layer)
        bi_lstm = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(128, dropout=0.3, recurrent_dropout=0.2, return_sequences=True)
        )(conv)
        drop = tf.keras.layers.Dropout(0.2)(bi_lstm)
        dense = tf.keras.layers.Dense(128, activation='relu')(drop)
        flat = tf.keras.layers.Flatten()(dense)
        output_layer = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(flat)

        model = tf.keras.Model(input_layer, output_layer)
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        # Train the model
        model.fit(
            X_train, y_train,
            epochs=5,  # You can adjust the number of epochs
            batch_size=32,
            verbose=0
        )

        # Evaluate the model
        y_pred = model.predict(X_val, verbose=0).argmax(axis=1)
        score = accuracy_score(y_val, y_pred)
        scores.append(score)

        # Generate classification report
        report = classification_report(y_val, y_pred, target_names=target_names, output_dict=True)
        reports.append(report)

        print(f"\nFold {fold + 1} Results:")
        print(classification_report(y_val, y_pred, target_names=target_names))

    return np.mean(scores), np.std(scores), reports

In [ ]:
aspects = ['Acting', 'direction', 'Music', 'Genre', 'excitement', 'ovr_sent']
sampling_techniques_under = ['undersample']
results = analyze_all_aspects_dl(df, aspects, sampling_techniques_under)


Analyzing aspect: Acting

Using undersample sampling:


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 1 Results:
              precision    recall  f1-score   support

          -1       0.05      0.62      0.09        34
           0       0.76      0.18      0.29       770
           1       0.25      0.46      0.32       221

    accuracy                           0.26      1025
   macro avg       0.35      0.42      0.23      1025
weighted avg       0.63      0.26      0.29      1025



/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 2 Results:
              precision    recall  f1-score   support

          -1       0.08      0.55      0.14        33
           0       0.75      0.39      0.51       769
           1       0.23      0.41      0.30       222

    accuracy                           0.40      1024
   macro avg       0.35      0.45      0.32      1024
weighted avg       0.62      0.40      0.45      1024



...


Fold 3 Results:
              precision    recall  f1-score   support

          -1       0.49      0.56      0.52       343
           0       0.24      0.53      0.33       153
           1       0.70      0.38      0.49       528

    accuracy                           0.46      1024
   macro avg       0.47      0.49      0.45      1024
weighted avg       0.56      0.46      0.48      1024



/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 4 Results:
              precision    recall  f1-score   support

          -1       0.49      0.49      0.49       344
           0       0.28      0.44      0.35       152
           1       0.68      0.57      0.62       528

    accuracy                           0.53      1024
   macro avg       0.49      0.50      0.49      1024
weighted avg       0.56      0.53      0.54      1024



/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 5 Results:
              precision    recall  f1-score   support

          -1       0.54      0.27      0.36       344
           0       0.29      0.36      0.32       152
           1       0.58      0.73      0.65       528

    accuracy                           0.52      1024
   macro avg       0.47      0.45      0.44      1024
weighted avg       0.53      0.52      0.50      1024

Mean Accuracy: 0.4927 (±0.0258)


In [ ]:
sampling_techniques_over = ['oversample']
results = analyze_all_aspects_dl(df, aspects, sampling_techniques_over)


Analyzing aspect: Acting

Using oversample sampling:


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 1 Results:
              precision    recall  f1-score   support

          -1       0.07      0.26      0.11        34
           0       0.80      0.59      0.68       770
           1       0.27      0.39      0.32       221

    accuracy                           0.54      1025
   macro avg       0.38      0.42      0.37      1025
weighted avg       0.66      0.54      0.58      1025



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 2 Results:
              precision    recall  f1-score   support

          -1       0.06      0.30      0.10        33
           0       0.78      0.63      0.70       769
           1       0.31      0.32      0.32       222

    accuracy                           0.55      1024
   macro avg       0.38      0.42      0.37      1024
weighted avg       0.66      0.55      0.59      1024



...

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 3 Results:
              precision    recall  f1-score   support

          -1       0.48      0.48      0.48       343
           0       0.26      0.49      0.34       153
           1       0.69      0.52      0.59       528

    accuracy                           0.50      1024
   macro avg       0.48      0.50      0.47      1024
weighted avg       0.56      0.50      0.52      1024



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 4 Results:
              precision    recall  f1-score   support

          -1       0.51      0.37      0.43       344
           0       0.21      0.64      0.32       152
           1       0.73      0.44      0.55       528

    accuracy                           0.44      1024
   macro avg       0.48      0.48      0.43      1024
weighted avg       0.58      0.44      0.47      1024



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


dravidian w2vec

In [ ]:
aspects = ['Acting', 'direction', 'Music', 'Genre', 'excitement', 'ovr_sent']
sampling_techniques_under = ['undersample']
results = analyze_all_aspects_dl(df, aspects, sampling_techniques_under)


Analyzing aspect: Acting

Using undersample sampling:


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 1 Results:
              precision    recall  f1-score   support

          -1       0.06      0.41      0.11        34
           0       0.87      0.64      0.74       770
           1       0.58      0.60      0.59       221

    accuracy                           0.62      1025
   macro avg       0.50      0.55      0.48      1025
weighted avg       0.78      0.62      0.68      1025



/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 2 Results:
              precision    recall  f1-score   support

          -1       0.09      0.58      0.15        33
           0       0.86      0.73      0.79       769
           1       0.65      0.45      0.53       222

    accuracy                           0.67      1024
   macro avg       0.53      0.59      0.49      1024
weighted avg       0.79      0.67      0.71      1024



...


Fold 3 Results:
              precision    recall  f1-score   support

          -1       0.79      0.43      0.56       343
           0       0.26      0.80      0.39       153
           1       0.89      0.61      0.72       528

    accuracy                           0.58      1024
   macro avg       0.64      0.61      0.56      1024
weighted avg       0.76      0.58      0.62      1024



/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 4 Results:
              precision    recall  f1-score   support

          -1       0.68      0.69      0.69       344
           0       0.36      0.53      0.43       152
           1       0.81      0.69      0.75       528

    accuracy                           0.67      1024
   macro avg       0.62      0.64      0.62      1024
weighted avg       0.70      0.67      0.68      1024



/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 5 Results:
              precision    recall  f1-score   support

          -1       0.69      0.75      0.71       344
           0       0.35      0.52      0.42       152
           1       0.85      0.68      0.75       528

    accuracy                           0.68      1024
   macro avg       0.63      0.65      0.63      1024
weighted avg       0.72      0.68      0.69      1024

Mean Accuracy: 0.6522 (±0.0379)


In [ ]:
sampling_techniques_over = ['oversample']
results = analyze_all_aspects_dl(df, aspects, sampling_techniques_over)


Analyzing aspect: Acting

Using oversample sampling:


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 1 Results:
              precision    recall  f1-score   support

          -1       0.07      0.15      0.10        34
           0       0.91      0.83      0.87       770
           1       0.59      0.67      0.62       221

    accuracy                           0.77      1025
   macro avg       0.52      0.55      0.53      1025
weighted avg       0.81      0.77      0.79      1025



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 2 Results:
              precision    recall  f1-score   support

          -1       0.10      0.30      0.15        33
           0       0.88      0.77      0.82       769
           1       0.57      0.64      0.60       222

    accuracy                           0.73      1024
   macro avg       0.52      0.57      0.52      1024
weighted avg       0.79      0.73      0.75      1024



...


Fold 3 Results:
              precision    recall  f1-score   support

          -1       0.70      0.60      0.65       343
           0       0.33      0.42      0.37       153
           1       0.79      0.80      0.79       528

    accuracy                           0.67      1024
   macro avg       0.61      0.61      0.60      1024
weighted avg       0.69      0.67      0.68      1024



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 4 Results:
              precision    recall  f1-score   support

          -1       0.67      0.69      0.68       344
           0       0.33      0.34      0.33       152
           1       0.81      0.79      0.80       528

    accuracy                           0.69      1024
   macro avg       0.60      0.60      0.60      1024
weighted avg       0.69      0.69      0.69      1024



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 5 Results:
              precision    recall  f1-score   support

          -1       0.68      0.77      0.73       344
           0       0.35      0.53      0.43       152
           1       0.86      0.66      0.75       528

    accuracy                           0.68      1024
   macro avg       0.63      0.66      0.63      1024
weighted avg       0.73      0.68      0.69      1024

Mean Accuracy: 0.6768 (±0.0083)


Dravidian Fastetxt

In [ ]:
aspects = ['Acting', 'direction', 'Music', 'Genre', 'excitement', 'ovr_sent']
sampling_techniques_under = ['undersample']
results = analyze_all_aspects_dl(df, aspects, sampling_techniques_under)


Analyzing aspect: Acting

Using undersample sampling:


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 1 Results:
              precision    recall  f1-score   support

          -1       0.07      0.56      0.13        34
           0       0.89      0.53      0.66       770
           1       0.48      0.67      0.56       221

    accuracy                           0.56      1025
   macro avg       0.48      0.59      0.45      1025
weighted avg       0.78      0.56      0.62      1025



/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 2 Results:
              precision    recall  f1-score   support

          -1       0.09      0.64      0.16        33
           0       0.92      0.50      0.65       769
           1       0.47      0.79      0.59       222

    accuracy                           0.56      1024
   macro avg       0.49      0.64      0.46      1024
weighted avg       0.80      0.56      0.62      1024



...


Fold 3 Results:
              precision    recall  f1-score   support

          -1       0.73      0.69      0.71       343
           0       0.33      0.56      0.42       153
           1       0.86      0.72      0.79       528

    accuracy                           0.69      1024
   macro avg       0.64      0.66      0.64      1024
weighted avg       0.74      0.69      0.71      1024



/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 4 Results:
              precision    recall  f1-score   support

          -1       0.66      0.75      0.70       344
           0       0.36      0.53      0.43       152
           1       0.87      0.67      0.76       528

    accuracy                           0.68      1024
   macro avg       0.63      0.65      0.63      1024
weighted avg       0.72      0.68      0.69      1024



/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 5 Results:
              precision    recall  f1-score   support

          -1       0.75      0.49      0.60       344
           0       0.27      0.68      0.39       152
           1       0.81      0.64      0.71       528

    accuracy                           0.60      1024
   macro avg       0.61      0.60      0.57      1024
weighted avg       0.71      0.60      0.63      1024

Mean Accuracy: 0.6382 (±0.0376)


In [ ]:
sampling_techniques_over = ['oversample']
results = analyze_all_aspects_dl(df, aspects, sampling_techniques_over)


Analyzing aspect: Acting

Using oversample sampling:


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 1 Results:
              precision    recall  f1-score   support

          -1       0.09      0.12      0.10        34
           0       0.89      0.91      0.90       770
           1       0.72      0.63      0.67       221

    accuracy                           0.82      1025
   macro avg       0.57      0.55      0.56      1025
weighted avg       0.83      0.82      0.83      1025



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 2 Results:
              precision    recall  f1-score   support

          -1       0.09      0.12      0.11        33
           0       0.85      0.90      0.87       769
           1       0.64      0.47      0.54       222

    accuracy                           0.78      1024
   macro avg       0.53      0.50      0.51      1024
weighted avg       0.78      0.78      0.78      1024



...


Fold 3 Results:
              precision    recall  f1-score   support

          -1       0.67      0.69      0.68       343
           0       0.31      0.37      0.34       153
           1       0.83      0.77      0.80       528

    accuracy                           0.69      1024
   macro avg       0.61      0.61      0.61      1024
weighted avg       0.70      0.69      0.69      1024



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 4 Results:
              precision    recall  f1-score   support

          -1       0.64      0.76      0.70       344
           0       0.31      0.49      0.38       152
           1       0.88      0.61      0.72       528

    accuracy                           0.64      1024
   macro avg       0.61      0.62      0.60      1024
weighted avg       0.71      0.64      0.66      1024



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 5 Results:
              precision    recall  f1-score   support

          -1       0.65      0.79      0.71       344
           0       0.36      0.44      0.40       152
           1       0.84      0.66      0.74       528

    accuracy                           0.67      1024
   macro avg       0.62      0.63      0.62      1024
weighted avg       0.70      0.67      0.68      1024

Mean Accuracy: 0.6657 (±0.0140)


In [ ]:
train_df = pd.read_csv("train_data.csv")
test_df = pd.read_csv("test_data.csv")

# Ensure EMBEDDING_DIM and embeddings_index are defined in your context

def train_final_dl_models_from_csv(train_df, test_df, aspects):
    """
    For each aspect, loads training data from train_df and testing data from test_df.
    Performs 5-fold cross validation on the training data to select the best deep learning model,
    averaging the training accuracy across folds and using class weights to address imbalanced datasets.
    Evaluates the best model on both the training and test sets and prints classification reports.

    Returns:
        best_models: dict of best models (with their label encoder and tokenizer) for each aspect.
        train_reports: dict of training classification reports for each aspect.
        test_reports: dict of test classification reports for each aspect.
    """
    best_models = {}
    train_reports = {}
    test_reports = {}
    maxlen = 15  # Maximum sequence length

    for aspect in aspects:
        print(f"\n=== Processing aspect: {aspect} ===")
        # Prepare the data for the current aspect
        X_train, y_train = prepare_data(train_df, aspect)
        X_test, y_test = prepare_data(test_df, aspect)

        # Encode target labels
        le = LabelEncoder()
        y_train_encoded = le.fit_transform(y_train)
        y_test_encoded = le.transform(y_test)  # Use the same encoder for test data
        target_names = [str(label) for label in le.classes_]

        # Tokenize the training texts and pad sequences
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(X_train['clean_text'])
        train_sequences = tokenizer.texts_to_sequences(X_train['clean_text'])
        test_sequences = tokenizer.texts_to_sequences(X_test['clean_text'])
        X_train_padded = pad_sequences(train_sequences, maxlen=maxlen)
        X_test_padded = pad_sequences(test_sequences, maxlen=maxlen)

        # Prepare the embedding matrix using the tokenizer's word index
        word_index = tokenizer.word_index
        num_words = len(word_index) + 1
        embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
        for word, i in word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                if len(embedding_vector) == EMBEDDING_DIM:
                    embedding_matrix[i] = embedding_vector
                else:
                    print(f"Warning: Embedding for '{word}' has unexpected dimension {len(embedding_vector)}. Skipping.")
        vocab_size = num_words

        # Cross-validation on the training data to select the best model
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        best_cv_score = -np.inf
        best_model = None
        cv_train_scores = []  # To store training accuracies for each fold

        for fold, (cv_train_idx, cv_val_idx) in enumerate(skf.split(X_train_padded, y_train_encoded)):
            X_cv_train = X_train_padded[cv_train_idx]
            y_cv_train = y_train_encoded[cv_train_idx]
            X_cv_val = X_train_padded[cv_val_idx]
            y_cv_val = y_train_encoded[cv_val_idx]

            # Compute class weights to address imbalance
            classes = np.unique(y_cv_train)
            cw = compute_class_weight(class_weight='balanced', classes=classes, y=y_cv_train)
            class_weight = dict(zip(classes, cw))

            # Define the model architecture
            input_layer = tf.keras.layers.Input(shape=(maxlen,))
            embedding_layer = tf.keras.layers.Embedding(
                vocab_size,
                EMBEDDING_DIM,
                weights=[embedding_matrix],
                trainable=False
            )(input_layer)
            conv = tf.keras.layers.Conv1D(200, kernel_size=3, activation="relu")(embedding_layer)
            bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, dropout=0.3, recurrent_dropout=0.2, return_sequences=True))(conv)
            drop = tf.keras.layers.Dropout(0.2)(bi_lstm)
            dense = tf.keras.layers.Dense(128, activation='relu',
                                          kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01))(drop)
            flat = tf.keras.layers.Flatten()(dense)
            output_layer = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(flat)

            model = tf.keras.Model(input_layer, output_layer)
            model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

            # Train the model on the current CV fold using class weights
            model.fit(X_cv_train, y_cv_train, epochs=5, batch_size=32, verbose=0, class_weight=class_weight)

            # Evaluate on the CV training fold for average training score
            y_cv_train_pred = model.predict(X_cv_train, verbose=0).argmax(axis=1)
            fold_train_accuracy = accuracy_score(y_cv_train, y_cv_train_pred)
            cv_train_scores.append(fold_train_accuracy)

            # Evaluate on the CV validation fold
            y_cv_val_pred = model.predict(X_cv_val, verbose=0).argmax(axis=1)
            cv_score = accuracy_score(y_cv_val, y_cv_val_pred)
            print(f"Fold {fold + 1} CV Accuracy: {cv_score:.4f}, Training Accuracy: {fold_train_accuracy:.4f}")

            # Select the best model based on CV validation accuracy
            if cv_score > best_cv_score:
                best_cv_score = cv_score
                best_model = model

        avg_cv_train_score = np.mean(cv_train_scores)
        print(f"Average Training Accuracy across folds: {avg_cv_train_score:.4f}")

        # Evaluate the best model on the entire training set
        y_train_pred = best_model.predict(X_train_padded, verbose=0).argmax(axis=1)
        train_acc = accuracy_score(y_train_encoded, y_train_pred)
        train_report = classification_report(y_train_encoded, y_train_pred, target_names=target_names)
        print(f"Best CV model Training Accuracy on full training set: {train_acc:.4f}")
        print("Training Classification Report:\n", train_report)

        # Evaluate on the test set
        y_test_pred = best_model.predict(X_test_padded, verbose=0).argmax(axis=1)
        test_acc = accuracy_score(y_test_encoded, y_test_pred)
        test_report = classification_report(y_test_encoded, y_test_pred, target_names=target_names)
        print(f"Test Accuracy: {test_acc:.4f}")
        print("Test Classification Report:\n", test_report)

        best_models[aspect.lower()] = {
            "model": best_model,
            "label_encoder": le,
            "tokenizer": tokenizer
        }
        train_reports[aspect.lower()] = train_report
        test_reports[aspect.lower()] = test_report

    return best_models, train_reports, test_reports


In [ ]:
aspects = ['Acting', 'direction', 'Music','ovr_sent']

In [ ]:
best_models, train_reports, test_reports = train_final_dl_models_from_csv(train_df,test_df, aspects)


=== Processing aspect: Acting ===
Fold 1 CV Accuracy: 0.7215, Training Accuracy: 0.7245
Fold 2 CV Accuracy: 0.5381, Training Accuracy: 0.5822
Fold 3 CV Accuracy: 0.6117, Training Accuracy: 0.6938
Fold 4 CV Accuracy: 0.7411, Training Accuracy: 0.7919
Fold 5 CV Accuracy: 0.5635, Training Accuracy: 0.6146
Fold 6 CV Accuracy: 0.7665, Training Accuracy: 0.8261
Fold 7 CV Accuracy: 0.7792, Training Accuracy: 0.8294
Fold 8 CV Accuracy: 0.7259, Training Accuracy: 0.8207
Fold 9 CV Accuracy: 0.5939, Training Accuracy: 0.6603
Fold 10 CV Accuracy: 0.8173, Training Accuracy: 0.8517
Average Training Accuracy across folds: 0.7395
Best CV model Training Accuracy on full training set: 0.8483
Training Classification Report:
               precision    recall  f1-score   support

          -1       0.44      0.80      0.57       131
           0       0.94      0.86      0.90      2962
           1       0.69      0.83      0.75       848

    accuracy                           0.85      3941
   macro av

In [ ]:
def prepare_data(df, aspect_column):
    """Prepare data for a specific aspect"""
    # Encode text to numerical values if needed
    # le = LabelEncoder()
    # Changed to return a DataFrame instead of a Series
    X = df[['R_clean_text']]
    y = df[aspect_column]
    return X, y

In [ ]:
train_df = pd.read_csv("Romanised_train_data.csv")
test_df = pd.read_csv("Romanised_test_data.csv")

# Ensure EMBEDDING_DIM and embeddings_index are defined in your context

def train_final_dl_models_from_csv(train_df, test_df, aspects):
    """
    For each aspect, loads training data from train_df and testing data from test_df.
    Performs 5-fold cross validation on the training data to select the best deep learning model,
    averaging the training accuracy across folds and using class weights to address imbalanced datasets.
    Evaluates the best model on both the training and test sets and prints classification reports.

    Returns:
        best_models: dict of best models (with their label encoder and tokenizer) for each aspect.
        train_reports: dict of training classification reports for each aspect.
        test_reports: dict of test classification reports for each aspect.
    """
    best_models = {}
    train_reports = {}
    test_reports = {}
    maxlen = 15  # Maximum sequence length

    for aspect in aspects:
        print(f"\n=== Processing aspect: {aspect} ===")
        # Prepare the data for the current aspect
        X_train, y_train = prepare_data(train_df, aspect)
        X_test, y_test = prepare_data(test_df, aspect)

        # Encode target labels
        le = LabelEncoder()
        y_train_encoded = le.fit_transform(y_train)
        y_test_encoded = le.transform(y_test)  # Use the same encoder for test data
        target_names = [str(label) for label in le.classes_]

        # Tokenize the training texts and pad sequences
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(X_train['R_clean_text'])
        train_sequences = tokenizer.texts_to_sequences(X_train['R_clean_text'])
        test_sequences = tokenizer.texts_to_sequences(X_test['R_clean_text'])
        X_train_padded = pad_sequences(train_sequences, maxlen=maxlen)
        X_test_padded = pad_sequences(test_sequences, maxlen=maxlen)

        # Prepare the embedding matrix using the tokenizer's word index
        word_index = tokenizer.word_index
        num_words = len(word_index) + 1
        embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
        for word, i in word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                if len(embedding_vector) == EMBEDDING_DIM:
                    embedding_matrix[i] = embedding_vector
                else:
                    print(f"Warning: Embedding for '{word}' has unexpected dimension {len(embedding_vector)}. Skipping.")
        vocab_size = num_words

        # Cross-validation on the training data to select the best model
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        best_cv_score = -np.inf
        best_model = None
        cv_train_scores = []  # To store training accuracies for each fold

        for fold, (cv_train_idx, cv_val_idx) in enumerate(skf.split(X_train_padded, y_train_encoded)):
            X_cv_train = X_train_padded[cv_train_idx]
            y_cv_train = y_train_encoded[cv_train_idx]
            X_cv_val = X_train_padded[cv_val_idx]
            y_cv_val = y_train_encoded[cv_val_idx]

            # Compute class weights to address imbalance
            classes = np.unique(y_cv_train)
            cw = compute_class_weight(class_weight='balanced', classes=classes, y=y_cv_train)
            class_weight = dict(zip(classes, cw))

            # Define the model architecture
            input_layer = tf.keras.layers.Input(shape=(maxlen,))
            embedding_layer = tf.keras.layers.Embedding(
                vocab_size,
                EMBEDDING_DIM,
                weights=[embedding_matrix],
                trainable=False
            )(input_layer)
            conv = tf.keras.layers.Conv1D(200, kernel_size=3, activation="relu")(embedding_layer)
            bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, dropout=0.3, recurrent_dropout=0.2, return_sequences=True))(conv)
            drop = tf.keras.layers.Dropout(0.2)(bi_lstm)
            dense = tf.keras.layers.Dense(128, activation='relu',
                                          kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01))(drop)
            flat = tf.keras.layers.Flatten()(dense)
            output_layer = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(flat)

            model = tf.keras.Model(input_layer, output_layer)
            model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

            # Train the model on the current CV fold using class weights
            model.fit(X_cv_train, y_cv_train, epochs=5, batch_size=32, verbose=0, class_weight=class_weight)

            # Evaluate on the CV training fold for average training score
            y_cv_train_pred = model.predict(X_cv_train, verbose=0).argmax(axis=1)
            fold_train_accuracy = accuracy_score(y_cv_train, y_cv_train_pred)
            cv_train_scores.append(fold_train_accuracy)

            # Evaluate on the CV validation fold
            y_cv_val_pred = model.predict(X_cv_val, verbose=0).argmax(axis=1)
            cv_score = accuracy_score(y_cv_val, y_cv_val_pred)
            print(f"Fold {fold + 1} CV Accuracy: {cv_score:.4f}, Training Accuracy: {fold_train_accuracy:.4f}")

            # Select the best model based on CV validation accuracy
            if cv_score > best_cv_score:
                best_cv_score = cv_score
                best_model = model

        avg_cv_train_score = np.mean(cv_train_scores)
        print(f"Average Training Accuracy across folds: {avg_cv_train_score:.4f}")

        # Evaluate the best model on the entire training set
        y_train_pred = best_model.predict(X_train_padded, verbose=0).argmax(axis=1)
        train_acc = accuracy_score(y_train_encoded, y_train_pred)
        train_report = classification_report(y_train_encoded, y_train_pred, target_names=target_names)
        print(f"Best CV model Training Accuracy on full training set: {train_acc:.4f}")
        print("Training Classification Report:\n", train_report)

        # Evaluate on the test set
        y_test_pred = best_model.predict(X_test_padded, verbose=0).argmax(axis=1)
        test_acc = accuracy_score(y_test_encoded, y_test_pred)
        test_report = classification_report(y_test_encoded, y_test_pred, target_names=target_names)
        print(f"Test Accuracy: {test_acc:.4f}")
        print("Test Classification Report:\n", test_report)

        best_models[aspect.lower()] = {
            "model": best_model,
            "label_encoder": le,
            "tokenizer": tokenizer
        }
        train_reports[aspect.lower()] = train_report
        test_reports[aspect.lower()] = test_report

    return best_models, train_reports, test_reports


In [ ]:
best_models, train_reports, test_reports = train_final_dl_models_from_csv(train_df,test_df, aspects)


=== Processing aspect: Acting ===
Fold 1 CV Accuracy: 0.6304, Training Accuracy: 0.6362
Fold 2 CV Accuracy: 0.7487, Training Accuracy: 0.8280
Fold 3 CV Accuracy: 0.5076, Training Accuracy: 0.5653
Fold 4 CV Accuracy: 0.6447, Training Accuracy: 0.7260
Fold 5 CV Accuracy: 0.7792, Training Accuracy: 0.8323
Fold 6 CV Accuracy: 0.8020, Training Accuracy: 0.8283
Fold 7 CV Accuracy: 0.5330, Training Accuracy: 0.5591
Fold 8 CV Accuracy: 0.6066, Training Accuracy: 0.7048
Fold 9 CV Accuracy: 0.6777, Training Accuracy: 0.7860
Fold 10 CV Accuracy: 0.6878, Training Accuracy: 0.7378
Average Training Accuracy across folds: 0.7204
Best CV model Training Accuracy on full training set: 0.8257
Training Classification Report:
               precision    recall  f1-score   support

          -1       0.32      0.67      0.43       131
           0       0.93      0.85      0.88      2962
           1       0.68      0.78      0.73       848

    accuracy                           0.83      3941
   macro av